#### What is Topic Modeling?

As the name suggests, it is a process to automatically identify topics present in a text object and to derive hidden patterns exhibited by a text corpus. Thus, assisting better decision making.

#### How it is different from rule-based text mining approch?

Rule based apprch use regular expressions or dictionary based keyword searching techniques where as Topic Modeling is an unsupervised approach used for finding and observing the bunch of words (called “topics”) in large clusters of texts.

#### 3 common technique of Topic Modeling


*   Latent Semantic Analysis (LSA)
*   Probabilistic Latent Semantic Analysis (pLSA)
*   Latent Dirichlet Allocation (LDA)





### LDA

In [2]:
!pip install pyLDAvis
!pip install pyLDAvis.gensim
!pip install django-model-utils

     |████████████████████████████████| 1.7 MB 2.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 24.0 MB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 40.7 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136900 sha256=18277c0f95a5c35f4c13da0cfa612dbe7e3943503ad656cbbde9c2df68efd0bf
  Stored in directory: /Users/swapnilkailassethi/Library/Caches/pip/wheels/57/a4/86/d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
Successfully built pyLDAvis
ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim
     |████████████████████████████████| 8.0 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 9.4 MB/s  eta 0:00:01


In [5]:
import pandas as pd
import numpy as np
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
#from model_utils import qc_dict, out_topics_docs, check_topic_doc_prob, topn_docs_by_topic
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [22]:
df =pd.read_csv('TV Ceiling & Wall Mounts_processed.csv')

In [23]:
df.head()

,asin,reviewText,overall,summary,title,review_no_html,review_no_contraction,review_no_punc,review_no_stopwords,review_lemmatized
0,972683275,"This is a great buy, compared to a $60 or more...",5,Excelant mount for Tv. Would buy more if I nee...,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,"This is a great buy, compared to a $60 or more...","This is a great buy, compared to a $60 or more...",This is a great buy compared to a 60 or more a...,great buy compared 60 retail store,great buy compare retail store
1,972683275,This mount is just what I needed. It is stron...,5,Perfect,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,This mount is just what I needed. It is stron...,This mount is just what I needed. It is stron...,This mount is just what I needed It is strong...,needed strong sturdy folds almost flat enable ...,need strong sturdy fold almost flat enable tel...
2,972683275,"Great deal, easy to mount and it appears to be...",5,32 In TV Mount,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,"Great deal, easy to mount and it appears to be...","Great deal, easy to mount and it appears to be...",Great deal easy to mount and it appears to be ...,great deal easy appears fairly sturdy mounted ...,great deal easy appear fairly sturdy work great
3,972683275,This mount works really well once you get it u...,4,"Pretty simple, but definitely good!",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,This mount works really well once you get it u...,This mount works really well once you get it u...,This mount works really well once you get it u...,works really well get going problem arms littl...,work really well get go problem arm little goo...
4,972683275,This wall mount does everything it's supposed ...,4,Fairly good wall mount,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,This wall mount does everything it's supposed ...,This wall mount does everything it is supposed...,This wall mount does everything it is supposed...,everything supposed piece attaches overly long...,everything suppose piece attache overly long m...


Create the Dictionary and Corpus needed for Topic Modeling

In [24]:
reviews = df['review_lemmatized'].copy()
reviews = reviews.apply(lambda x: x.split())

In [25]:
reviews

0                     [great, buy, compare, retail, store]
1        [need, strong, sturdy, fold, almost, flat, ena...
2        [great, deal, easy, appear, fairly, sturdy, wo...
3        [work, really, well, get, go, problem, arm, li...
4        [everything, suppose, piece, attache, overly, ...
                               ...                        
13114    [reason, go, one, sometimes, remote, hard, tim...
13115    [really, well, make, extremely, durable, set, ...
13116    [great, little, contraption, keep, apple, stat...
13117    [sturdy, affix, supply, double, sided, tape, h...
13118    [wellmade, accessory, hold, apple, device, sec...
Name: review_lemmatized, Length: 13119, dtype: object

In [26]:
# Create Dictionary
dictionary = corpora.Dictionary(reviews)
dictionary.filter_extremes(no_below=3)

In [27]:
# number of terms
nd = dictionary.num_docs
nt = len(dictionary.keys())
print("number of documents", nd)
print("number of terms", nt)

number of documents 13119
number of terms 5707


In [28]:
# Create Corpus
texts = reviews

# Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [29]:
# Human readable format of corpus (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('buy', 1), ('compare', 1), ('great', 1), ('retail', 1), ('store', 1)]]

Building the Topic Model

In [30]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           iterations = 400)

View the topics in LDA model

In [31]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.084*"drywall" + 0.084*"behind" + 0.074*"range" + 0.060*"plug" + '
  '0.054*"hide" + 0.040*"super" + 0.040*"call" + 0.036*"send" + 0.035*"finder" '
  '+ 0.034*"durable"'),
 (1,
  '0.121*"use" + 0.079*"screw" + 0.058*"bracket" + 0.054*"bolt" + 0.049*"stud" '
  '+ 0.043*"two" + 0.037*"level" + 0.036*"include" + 0.034*"small" + '
  '0.029*"attach"'),
 (2,
  '0.050*"much" + 0.045*"installation" + 0.044*"instal" + 0.043*"set" + '
  '0.041*"unit" + 0.038*"think" + 0.037*"right" + 0.035*"instruction" + '
  '0.035*"enough" + 0.034*"place"'),
 (3,
  '0.097*"work" + 0.085*"easy" + 0.077*"great" + 0.071*"good" + 0.063*"well" + '
  '0.055*"install" + 0.051*"buy" + 0.051*"hold" + 0.044*"price" + '
  '0.034*"sturdy"'),
 (4,
  '0.090*"screen" + 0.090*"flat" + 0.053*"vesa" + 0.048*"low" + 0.046*"x" + '
  '0.045*"view" + 0.041*"plasma" + 0.041*"turn" + 0.039*"space" + 0.036*"mm"'),
 (5,
  '0.424*"hole" + 0.214*"drill" + 0.067*"measure" + 0.049*"concrete" + '
  '0.047*"apart" + 0.033*"write" + 

#### Chages on each run

Topic 0 is a represented as _0.082*"minute" + 0.031*"player" + 0.025*"customer" + 0.023*"dvd" + '
  '0.022*"hd" + 0.022*"inside" + 0.018*"beat" + 0.016*"double" + 0.016*"ne" + '
  '0.015*"function"


It means the top 10 keywords that contribute to this topic are: minute, player, customer.. and so on and the weight of minute on topic 0 is 0.082.

The weights reflect how important a keyword is to that topic.

 Compute Model Perplexity and Coherence Score

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.28093983685892

Coherence Score:  0.3123686289502873


Visualize the topics-keywords

In [33]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [34]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis


/Users/swapnilkailassethi/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/swapnilkailassethi/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/swapnilkailassethi/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.279489 -0.172574       1        1  36.701142
1      0.238962 -0.093083       2        1  14.696471
3      0.246156 -0.142399       3        1  14.557018
2      0.148505  0.208290       4        1   9.420748
9      0.075225  0.258900       5        1   5.290339
4      0.041025  0.182488       6        1   3.707109
17     0.031519  0.134618       7        1   2.884288
0     -0.021925  0.041901       8        1   1.760366
13    -0.039431  0.015260       9        1   1.240405
5     -0.066325 -0.018285      10        1   1.184967
6     -0.069355 -0.021846      11        1   1.132175
10    -0.066792 -0.018718      12        1   1.091288
11    -0.088418 -0.040302      13        1   1.071458
15    -0.075197 -0.027775      14        1   1.048505
8     -0.093458 -0.044827      15        1   0.983345
19    -0.062080 -0.013519      16        1   0.944777
16    -0.104439 -0.053277      17        1   0.803319
12    -0.102835 -0.052102      18        1   0.696987
14    -0.131197 -0.069665      19        1   0.467773
18    -0.139431 -0.073085      20        1   0.317520, topic_info=            Term         Freq        Total Category  logprob  loglift
125          use  9683.000000  9683.000000  Default  30.0000  30.0000
176         inch  3065.000000  3065.000000  Default  29.0000  29.0000
31          work  7205.000000  7205.000000  Default  28.0000  28.0000
29          easy  6324.000000  6324.000000  Default  27.0000  27.0000
39          hole  2554.000000  2554.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2187         usb     0.043285     1.184471  Topic20 -10.5265   2.4431
1240  experience     0.043284     1.184465  Topic20 -10.5265   2.4431
941          run     0.043287     1.184640  Topic20 -10.5264   2.4430
2983     network     0.043283     1.184464  Topic20 -10.5265   2.4431
1307        port     0.043283     1.184424  Topic20 -10.5265   2.4431

[849 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
127       1  0.999304      able
644       7  0.992020    actual
426       2  0.999035  actually
185       3  0.998734       add
190       7  0.999374    adjust
...     ...       ...       ...
149       4  0.000289     would
68       13  0.998279    wrench
742      10  0.994148     write
184       6  0.999070         x
703      16  0.994763       yet

[446 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 4, 3, 10, 5, 18, 1, 14, 6, 7, 11, 12, 16, 9, 20, 17, 13, 15, 19])

Interpretation: 
Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

Alright, if we move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.

 Building LDA Mallet Model

Upnext, we will improve upon this model by using Mallet’s version of LDA algorithm and then we will focus on how to arrive at the optimal number of topics given any large corpus of text.

     |████████████████████████████████| 23.4 MB 70.2 MB/s eta 0:00:01
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-macosx_10_9_x86_64.whl size=24175443 sha256=1f5f866c11c0e573c7b21addd02b7665390d75a43f490ec54b9e15a3608ad0ef
  Stored in directory: /Users/swapnilkailassethi/Library/Caches/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


ImportError: cannot import name 'Vocab' from 'gensim.models.word2vec' (/Users/swapnilkailassethi/opt/anaconda3/lib/python3.9/site-packages/gensim/models/word2vec.py)

AttributeError: module 'gensim.models' has no attribute 'wrappers'